In [3]:
import os
os.environ['CUDA_HOME'] = '/usr/lib/cuda'
os.environ['CUDA_PATH'] = '/usr/lib/cuda/bin'

In [4]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())

True
1


In [5]:
import copy
import json
import os
from pathlib import Path
import sys
import warnings
from anndata import AnnData
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import pandas as pd
import tqdm
import gseapy as gp

from torchtext.vocab import Vocab
from torchtext._torchtext import (
    Vocab as VocabPybind,
)

sys.path.insert(0, "../")
import scgpt as scg
from scgpt.tasks import GeneEmbedding
from scgpt.tokenizer.gene_tokenizer import GeneVocab
from scgpt.model import TransformerModel
from scgpt.preprocess import Preprocessor
from scgpt.utils import set_seed 

os.environ["KMP_WARNINGS"] = "off"
warnings.filterwarnings('ignore')

/home/dostofizky/anaconda3/envs/nrnb/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/dostofizky/anaconda3/envs/nrnb/lib/python3.10/site-packages/scgpt/model/model.py:21: UserWarning: flash_attn is not installed
  warnings.warn("flash_attn is not installed")
/home/dostofizky/anaconda3/envs/nrnb/lib/python3.10/site-packages/scgpt/model/multiomic_model.py:19: UserWarning: flash_attn is not installed
  warnings.warn("flash_attn is not installed")


In [7]:
set_seed(42)
pad_token = "<pad>"
special_tokens = [pad_token, "<cls>", "<eoc>"]
n_hvg = 1200
n_bins = 51
mask_value = -1
pad_value = -2
n_input_bins = n_bins

# Step 1: Load pre-trained model and dataset
## 1.1 Load pre-trained model

In [5]:
# Specify model path; here we load the pre-trained scGPT blood model
model_dir = Path("/home/dostofizky/Documents/NRNB238/models/scGPT_all_model/scGPT_human")
model_config_file = model_dir / "args.json"
model_file = model_dir / "best_model.pt"
vocab_file = model_dir / "vocab.json"

vocab = GeneVocab.from_file(vocab_file)
for s in special_tokens:
    if s not in vocab:
        vocab.append_token(s)

# Retrieve model parameters from config files
with open(model_config_file, "r") as f:
    model_configs = json.load(f)
print(
    f"Resume model from {model_file}, the model args will override the "
    f"config {model_config_file}."
)
embsize = model_configs["embsize"]
nhead = model_configs["nheads"]
d_hid = model_configs["d_hid"]
nlayers = model_configs["nlayers"]
n_layers_cls = model_configs["n_layers_cls"]

gene2idx = vocab.get_stoi()

Resume model from /home/dostofizky/Documents/NRNB238/models/scGPT_all_model/scGPT_human/best_model.pt, the model args will override the config /home/dostofizky/Documents/NRNB238/models/scGPT_all_model/scGPT_human/args.json.


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ntokens = len(vocab)  # size of vocabulary
model = TransformerModel(
    ntokens,
    embsize,
    nhead,
    d_hid,
    nlayers,
    vocab=vocab,
    pad_value=pad_value,
    n_input_bins=n_input_bins,
)

try:
    model.load_state_dict(torch.load(model_file))
    print(f"Loading all model params from {model_file}")
except:
    # only load params that are in the model and match the size
    model_dict = model.state_dict()
    pretrained_dict = torch.load(model_file)
    pretrained_dict = {
        k: v
        for k, v in pretrained_dict.items()
        if k in model_dict and v.shape == model_dict[k].shape
    }
    for k, v in pretrained_dict.items():
        print(f"Loading params {k} with shape {v.shape}")
        model_dict.update(pretrained_dict)
        model.load_state_dict(model_dict)

model.to(device)

Loading params encoder.embedding.weight with shape torch.Size([60697, 512])
Loading params encoder.enc_norm.weight with shape torch.Size([512])
Loading params encoder.enc_norm.bias with shape torch.Size([512])
Loading params value_encoder.linear1.weight with shape torch.Size([512, 1])
Loading params value_encoder.linear1.bias with shape torch.Size([512])
Loading params value_encoder.linear2.weight with shape torch.Size([512, 512])
Loading params value_encoder.linear2.bias with shape torch.Size([512])
Loading params value_encoder.norm.weight with shape torch.Size([512])
Loading params value_encoder.norm.bias with shape torch.Size([512])
Loading params transformer_encoder.layers.0.self_attn.out_proj.weight with shape torch.Size([512, 512])
Loading params transformer_encoder.layers.0.self_attn.out_proj.bias with shape torch.Size([512])
Loading params transformer_encoder.layers.0.linear1.weight with shape torch.Size([512, 512])
Loading params transformer_encoder.layers.0.linear1.bias with 

TransformerModel(
  (encoder): GeneEncoder(
    (embedding): Embedding(60697, 512, padding_idx=60694)
    (enc_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
  (value_encoder): ContinuousValueEncoder(
    (dropout): Dropout(p=0.5, inplace=False)
    (linear1): Linear(in_features=1, out_features=512, bias=True)
    (activation): ReLU()
    (linear2): Linear(in_features=512, out_features=512, bias=True)
    (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-11): 12 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
        (linear1): Linear(in_features=512, out_features=512, bias=True)
        (dropout): Dropout(p=0.5, inplace=False)
        (linear2): Linear(in_features=512, out_features=512, bias=True)
        (norm1): LayerNorm((512,), eps=1e-05, el

# 1.2 Load dataset of interest
## The Immune Human dataset can be downloaded via this link.

In [7]:
# Specify data path; here we load the Immune Human dataset
data_dir = Path("../data")
adata = sc.read(
    str(data_dir / "Immune_ALL_human.h5ad"), cache=True
)  # 33506 × 12303
ori_batch_col = "batch"
adata.obs["celltype"] = adata.obs["final_annotation"].astype(str)
data_is_raw = False

In [8]:
# Preprocess the data following the scGPT data pre-processing pipeline
preprocessor = Preprocessor(
    use_key="X",  # the key in adata.layers to use as raw data
    filter_gene_by_counts=3,  # step 1
    filter_cell_by_counts=False,  # step 2
    normalize_total=1e4,  # 3. whether to normalize the raw data and to what sum
    result_normed_key="X_normed",  # the key in adata.layers to store the normalized data
    log1p=data_is_raw,  # 4. whether to log1p the normalized data
    result_log1p_key="X_log1p",
    subset_hvg=n_hvg,  # 5. whether to subset the raw data to highly variable genes
    hvg_flavor="seurat_v3" if data_is_raw else "cell_ranger",
    binning=n_bins,  # 6. whether to bin the raw data and to what number of bins
    result_binned_key="X_binned",  # the key in adata.layers to store the binned data
)
preprocessor(adata, batch_key="batch")

scGPT - INFO - Filtering genes by counts ...
scGPT - INFO - Normalizing total counts ...
scGPT - INFO - Subsetting highly variable genes ...
scGPT - INFO - Binning data ...


# Step 2: Retrieve scGPT's gene embeddings

Note that technically scGPT's gene embeddings are data independent. Overall, the pre-trained foundation model contains 30+K genes. Here for simplicity, we focus on a subset of HVGs specific to the data at hand.

Then we save the Gene Embeddings in a .csv file to feed into the following algorithms in Beeline:
1. PIDC
2. GRNBoost2
3. GENIE3


In [9]:
# Retrieve the data-independent gene embeddings from scGPT
gene_ids = np.array([id for id in gene2idx.values()])
gene_embeddings = model.encoder(torch.tensor(gene_ids, dtype=torch.long).to(device))
gene_embeddings = gene_embeddings.detach().cpu().numpy()

In [10]:
# Filter on the intersection between the Immune Human HVGs found in step 1.2 and scGPT's 30+K foundation model vocab
gene_embeddings = {gene: gene_embeddings[i] for i, gene in enumerate(gene2idx.keys()) if gene in adata.var.index.tolist()}
print('Retrieved gene embeddings for {} genes.'.format(len(gene_embeddings)))

Retrieved gene embeddings for 1170 genes.


In [11]:
# Construct gene embedding network
embed = GeneEmbedding(gene_embeddings)

100%|██████████| 1170/1170 [00:00<00:00, 2217503.70it/s]


### Generates Numpy arrays of the gene embeddings

In [12]:
import pandas as pd

# Create a dictionary to store the gene embeddings
gene_embeddings_dict = {}

# Iterate over the gene embeddings
for gene, embedding in gene_embeddings.items():
    gene_embeddings_dict[gene] = embedding.tolist()

# Create a pandas DataFrame from the dictionary
gene_embeddings_df = pd.DataFrame.from_dict(gene_embeddings_dict, orient='index')

# Save the DataFrame as a CSV file
gene_embeddings_df.to_csv('gene_embeddings.csv')

### Generate Gene names as column headings

In [16]:
import pandas as pd

# Create a dictionary to store the gene embeddings
gene_embeddings_dict = {}

# Iterate over the gene embeddings
for gene, embedding in gene_embeddings.items():
    gene_embeddings_dict[gene] = embedding

# Create a pandas DataFrame from the dictionary
gene_embeddings_df = pd.DataFrame.from_dict(gene_embeddings_dict, orient='index')

# Save the DataFrame as a space-separated CSV file with gene names as row indices
gene_embeddings_df.to_csv('gene_embeddings_names_space_Separated.csv', sep=' ', header=False, index=True)

### Gene Embeddings with labels

In [14]:
import pandas as pd

# Create a dictionary to store the gene embeddings
gene_embeddings_dict = {}

# Iterate over the gene embeddings
for gene, embedding in gene_embeddings.items():
    gene_embeddings_dict[gene] = embedding

# Read the first row from the provided file
with open('input_file.txt', 'r') as f:
    labels = f.readline().strip().split('\t')

# Create a pandas DataFrame from the dictionary
gene_embeddings_df = pd.DataFrame.from_dict(gene_embeddings_dict, orient='index')

# Set the column names using the labels
gene_embeddings_df.columns = labels

# Save the DataFrame as a CSV file with labels as column names
gene_embeddings_df.to_csv('gene_embeddings_withLables.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'input_file.txt'

In [15]:
import csv
import os

# Function to read CSV file, replace commas with spaces, and overwrite the original file
def replace_commas_with_spaces(input_file):
    temp_output_file = 'temp_output.csv'  # Temporary file to hold modified content

    with open(input_file, 'r', newline='') as infile, open(temp_output_file, 'w', newline='') as outfile:
        reader = csv.reader(infile)
        writer = csv.writer(outfile, delimiter=' ')

        for row in reader:
            # Replace commas with spaces in each row
            new_row = [entry.replace(',', ' ') for entry in row]
            writer.writerow(new_row)

    # Replace original file with temporary output file
    os.replace(temp_output_file, input_file)
    print(f"CSV file '{input_file}' has been processed and overwritten with modified content.")

# Example usage
input_filename = 'gene_embeddings_names.csv'   # Replace with your input CSV file name

replace_commas_with_spaces(input_filename)

CSV file 'gene_embeddings_names.csv' has been processed and overwritten with modified content.


# Blood model x Blood dataset work

In [8]:
# Specify model path; here we load the pre-trained scGPT blood model
model_dir = Path("../models/scGPT_Blood_model/scGPT_bc")
model_config_file = model_dir / "args.json"
model_file = model_dir / "best_model.pt"
vocab_file = model_dir / "vocab.json"

vocab = GeneVocab.from_file(vocab_file)
for s in special_tokens:
    if s not in vocab:
        vocab.append_token(s)

# Retrieve model parameters from config files
with open(model_config_file, "r") as f:
    model_configs = json.load(f)
print(
    f"Resume model from {model_file}, the model args will override the "
    f"config {model_config_file}."
)
embsize = model_configs["embsize"]
nhead = model_configs["nheads"]
d_hid = model_configs["d_hid"]
nlayers = model_configs["nlayers"]
n_layers_cls = model_configs["n_layers_cls"]

gene2idx = vocab.get_stoi()

Resume model from ../models/scGPT_Blood_model/scGPT_bc/best_model.pt, the model args will override the config ../models/scGPT_Blood_model/scGPT_bc/args.json.


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ntokens = len(vocab)  # size of vocabulary
model = TransformerModel(
    ntokens,
    embsize,
    nhead,
    d_hid,
    nlayers,
    vocab=vocab,
    pad_value=pad_value,
    n_input_bins=n_input_bins,
)

try:
    model.load_state_dict(torch.load(model_file))
    print(f"Loading all model params from {model_file}")
except:
    # only load params that are in the model and match the size
    model_dict = model.state_dict()
    pretrained_dict = torch.load(model_file)
    pretrained_dict = {
        k: v
        for k, v in pretrained_dict.items()
        if k in model_dict and v.shape == model_dict[k].shape
    }
    for k, v in pretrained_dict.items():
        print(f"Loading params {k} with shape {v.shape}")
        model_dict.update(pretrained_dict)
        model.load_state_dict(model_dict)

model.to(device)

Loading params encoder.embedding.weight with shape torch.Size([36574, 512])
Loading params encoder.enc_norm.weight with shape torch.Size([512])
Loading params encoder.enc_norm.bias with shape torch.Size([512])
Loading params value_encoder.linear1.weight with shape torch.Size([512, 1])
Loading params value_encoder.linear1.bias with shape torch.Size([512])
Loading params value_encoder.linear2.weight with shape torch.Size([512, 512])
Loading params value_encoder.linear2.bias with shape torch.Size([512])
Loading params value_encoder.norm.weight with shape torch.Size([512])
Loading params value_encoder.norm.bias with shape torch.Size([512])
Loading params transformer_encoder.layers.0.self_attn.out_proj.weight with shape torch.Size([512, 512])
Loading params transformer_encoder.layers.0.self_attn.out_proj.bias with shape torch.Size([512])
Loading params transformer_encoder.layers.0.linear1.weight with shape torch.Size([512, 512])
Loading params transformer_encoder.layers.0.linear1.bias with 

TransformerModel(
  (encoder): GeneEncoder(
    (embedding): Embedding(36574, 512, padding_idx=36571)
    (enc_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
  (value_encoder): ContinuousValueEncoder(
    (dropout): Dropout(p=0.5, inplace=False)
    (linear1): Linear(in_features=1, out_features=512, bias=True)
    (activation): ReLU()
    (linear2): Linear(in_features=512, out_features=512, bias=True)
    (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-11): 12 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
        (linear1): Linear(in_features=512, out_features=512, bias=True)
        (dropout): Dropout(p=0.5, inplace=False)
        (linear2): Linear(in_features=512, out_features=512, bias=True)
        (norm1): LayerNorm((512,), eps=1e-05, el

In [22]:
from scipy import sparse

file_path = '../BEELINE-data/inputs/scRNA-Seq/mHSC-E/ExpressionData.csv'
def load_and_preprocess_data(file_path):
    df = pd.read_csv(file_path, index_col=0)
    genes = df.index.tolist()
    cells = df.columns.tolist()
    expression_matrix = df.values.astype(float)  # Ensure values are floats
    return genes, cells, expression_matrix

In [23]:
def prepare_data_for_scgpt(genes, expression_matrix, gene2idx, n_input_bins=51):
    data = []
    unk_id = gene2idx.get('<unk>', len(gene2idx))  # Use the next available index if '<unk>' is not present
    
    for cell_expr in expression_matrix.T:  # Transpose to iterate over cells
        nonzero_idx = np.nonzero(cell_expr)[0]
        values = cell_expr[nonzero_idx]
        
        genes_cell = [genes[i] for i in nonzero_idx]
        counts = values
        
        gene_ids = [gene2idx.get(gene, unk_id) for gene in genes_cell]
        
        # Normalize counts to input bins
        norm_counts = (counts / np.max(counts) * (n_input_bins - 1)).astype(int) + 1
        norm_counts = np.clip(norm_counts, 1, n_input_bins - 1)
        
        data.append({
            'gene_ids': gene_ids,
            'norm_counts': norm_counts.tolist()
        })
    return data

In [28]:
def generate_embeddings(model, data, device):
    model.eval()
    embeddings = []
    
    with torch.no_grad():
        for cell in data:
            gene_ids = torch.tensor(cell['gene_ids'], dtype=torch.long).unsqueeze(0).to(device)
            norm_counts = torch.tensor(cell['norm_counts'], dtype=torch.long).unsqueeze(0).to(device)
            
            # Create src_key_padding_mask
            src_key_padding_mask = torch.zeros_like(gene_ids, dtype=torch.bool).to(device)
            
            output = model(gene_ids, norm_counts, src_key_padding_mask=src_key_padding_mask)
            embedding = output.mean(dim=1)  # Average over gene dimension
            embeddings.append(embedding.cpu().numpy())
    
    return np.vstack(embeddings)

In [26]:
def generate_embeddings(model, data, device):
    model.eval()
    embeddings = []
    
    with torch.no_grad():
        for i, cell in enumerate(data):
            try:
                gene_ids = torch.tensor(cell['gene_ids'], dtype=torch.long).unsqueeze(0).to(device)
                norm_counts = torch.tensor(cell['norm_counts'], dtype=torch.long).unsqueeze(0).to(device)
                
                src_key_padding_mask = torch.zeros_like(gene_ids, dtype=torch.bool).to(device)
                
                output = model(gene_ids, norm_counts, src_key_padding_mask=src_key_padding_mask)
                embedding = output.mean(dim=1)
                embeddings.append(embedding.cpu().numpy())
            except Exception as e:
                print(f"Error processing cell {i}: {e}")
                print(f"gene_ids shape: {gene_ids.shape}, norm_counts shape: {norm_counts.shape}")
                raise
    
    return np.vstack(embeddings)

In [29]:
genes, cells, expression_matrix = load_and_preprocess_data(file_path)
data = prepare_data_for_scgpt(genes, expression_matrix, gene2idx, n_input_bins)

embeddings = generate_embeddings(model, data, device)

# Now 'embeddings' contains the gene embeddings for each cell
print(f"Generated embeddings shape: {embeddings.shape}")

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
